# 明暗タスクデータセット 要約


## 前処理
### Pythonによるファイル統合
心理学実験ソフトウェアのログをedfファイルに書き込む

In [ ]:
import os
from labedf import csv2,edf2,set2
import numpy as np
from scipy import signal
from pyedflib import EdfReader
from utils import edf as myedf,edflab as myedflab,signals_standardization,lowpass_filter
DATASET_DIR_PATH = "./dataset/lord2/train"
file_settings = myedflab.MergeAllCsv2EdfFileSettings(DATASET_DIR_PATH + "/ペア.csv",list_encoding="ansi")
edfcsv_filenames = file_settings.get_edfcsv_filenames()
with EdfReader(f"{DATASET_DIR_PATH}/edf/{edfcsv_filenames[0,0]}") as er:
    fs = int(myedf.get_fs(er))

# %% merge csv,edf
file_settings.build_dir_path = f"{file_settings.root_path}/build/1"
filenames = myedflab.merge_all_csv2edf(file_settings,label_header_name="LorD",marker_names=["Marker"],marker_offset=None)
file_settings.build_dir_path = f"{file_settings.root_path}/build/2"
_ = myedflab.merge_all_csv2edf(file_settings,label_header_name="LorD",marker_names=["Wait"],marker_offset=None)

### MATLABで前処理
+ [コード - コミット固定](https://github.com/s-n-1-0/sist_eeg_tests/blob/64c722b010dc63870cb3341b225c3c5873d027e8/notes/dataset/lord2/preprocessing.m)  
(変更点 : エポックリジェクション前の途中経過を保存する処理を加えた、待機時脳波も前処理対象に加えた)

## データセットサイズ

In [ ]:
import mne
labels = ["dark","light"]
before_wait_dataset = []
before_lord_dataset = []
after_wait_dataset = []
after_lord_dataset = []
# 刺激脳波
for filename in filenames:
    epochs = mne.io.read_epochs_eeglab(DATASET_DIR_PATH +"/snapshot/1/"+filename+".set")
    _ld = []
    for label in labels:
        _ld.append(epochs.get_data(item=f"Marker__{label}"))
    before_lord_dataset.append(_ld)
    epochs = mne.io.read_epochs_eeglab(DATASET_DIR_PATH +"/pre2/1/"+filename+".set")
    _ld = []
    for label in labels:
        _ld.append(epochs.get_data(item=f"Marker__{label}"))
    after_lord_dataset.append(_ld)

# 待機脳波
for filename in filenames:
    epochs = mne.io.read_epochs_eeglab(DATASET_DIR_PATH +"/snapshot/2/"+filename+".set")
    _wd = []
    for label in labels:
        _wd.append(epochs.get_data(item=f"Wait__{label}"))
    before_wait_dataset.append(np.concatenate(_wd)) # dark,lightにこだわりがないため結合
    epochs = mne.io.read_epochs_eeglab(DATASET_DIR_PATH +"/pre2/2/"+filename+".set")
    _wd = []
    for label in labels:
        _wd.append(epochs.get_data(item=f"Wait__{label}"))
    after_wait_dataset.append(np.concatenate(_wd))



In [ ]:
import matplotlib.pyplot as plt

before_wait_count = sum([wd.shape[0] for wd in before_wait_dataset])
before_dark_count = sum([ld[0].shape[0]   for ld in before_lord_dataset])
before_light_count = sum([ld[1].shape[0]   for ld in before_lord_dataset])

after_wait_count = sum([wd.shape[0] for wd in after_wait_dataset])
after_dark_count = sum([ld[0].shape[0]   for ld in after_lord_dataset])
after_light_count = sum([ld[1].shape[0]   for ld in after_lord_dataset])


print(f"前処理前 待機画像数 : {before_wait_count}")
print(f"前処理前 刺激画像数(暗) : {before_dark_count}")
print(f"前処理前 刺激画像数(明) : {before_light_count}")
print("------")
print(f"前処理後 待機画像数 : {after_wait_count}")
print(f"前処理後 刺激画像数(暗) : {after_dark_count}")
print(f"前処理後 刺激画像数(明) : {after_light_count}")

plt.bar(["wait"] + labels,[before_wait_count,before_dark_count,before_light_count])
plt.title("Before")
plt.show()
plt.title("After")
plt.bar(["wait"] + labels,[after_wait_count,after_dark_count,after_light_count])


## 加算平均・平均・標準偏差・標準誤差

In [ ]:
before_erp_wait = np.array([np.sum(wd,axis=0) / wd.shape[0] for wd in before_wait_dataset])
before_erp_dark = np.array([np.sum(ld[0],axis=0) / ld[0].shape[0]   for ld in before_lord_dataset])
before_erp_light = np.array([np.sum(ld[1],axis=0) / ld[1].shape[0]   for ld in before_lord_dataset])

after_erp_wait =  np.array([np.sum(wd,axis=0) / wd.shape[0] for wd in after_wait_dataset])
after_erp_dark = np.array([np.sum(ld[0],axis=0) / ld[0].shape[0]   for ld in after_lord_dataset])
after_erp_light = np.array([np.sum(ld[1],axis=0) / ld[1].shape[0]   for ld in after_lord_dataset])


### 全ファイルの合算について

In [ ]:
t_range = [i/500 - 1 for i in range(1500)]
def plot_all_ch(title,data:np.ndarray):
    fig = plt.figure(figsize=(20,8))
    fig.suptitle(title)
    plt.subplots_adjust(wspace=0.4, hspace=0.6)
    for i in range(10):
        ax = fig.add_subplot(3, 4, i+1)
        ax.plot(t_range,data[i,:])
        ax.set_title(f"{i + 1}ch")
before_all_erp_wait = np.sum(before_erp_wait,axis=0) / before_erp_wait.shape[0]
before_all_erp_dark = np.sum(before_erp_dark,axis=0) / before_erp_dark.shape[0]
before_all_erp_light = np.sum(before_erp_light,axis=0) / before_erp_light.shape[0]

after_all_erp_wait = np.sum(after_erp_wait,axis=0) / after_erp_wait.shape[0]
after_all_erp_dark = np.sum(after_erp_dark,axis=0) / after_erp_dark.shape[0]
after_all_erp_light = np.sum(after_erp_light,axis=0) / after_erp_light.shape[0]
plot_all_ch("[Before] All Mean Wait ERP",before_all_erp_wait)
plot_all_ch("[Before] All Mean Dark ERP",before_all_erp_dark)
plot_all_ch("[Before] All Mean Light ERP",before_all_erp_light)
plot_all_ch("[After] All Mean Wait ERP",after_all_erp_wait)
plot_all_ch("[After] All Mean Dark ERP",after_all_erp_dark)
plot_all_ch("[After] All Mean Light ERP",after_all_erp_light)

### 各ファイルの結果について
画像ファイルの数が多すぎるのでOnedriveで共有します。